# Arcfelismerés

Az adatok az LFW nyílt online adathalmazból származnak, amelyeket a scikit-learn fetch függvénnyel érhetünk el.

## 1.Feladat

In [2]:
from sklearn.datasets import fetch_lfw_people

lfw_dataset = fetch_lfw_people(min_faces_per_person=5, resize=1.0, color=False)
print("kesz")

kesz


In [3]:
print("Képek száma:", len(lfw_dataset.images))
print("Kép mérete (magasság, szélesség):", lfw_dataset.images.shape[1], "x", lfw_dataset.images.shape[2])
print("Célok száma:", len(lfw_dataset.target))
print("Személyek száma:", len(lfw_dataset.target_names))
print("Személyek nevei:", lfw_dataset.target_names)

Képek száma: 4254
Kép mérete (magasság, szélesség): 125 x 94
Célok száma: 4254
Személyek száma: 229
Személyek nevei: ['Al Gore' 'Alan Greenspan' 'Albert Costa' 'Alejandro Toledo' 'Ali Naimi'
 'Alvaro Uribe' 'Amelie Mauresmo' 'Ana Guevara' 'Ana Palacio'
 'Andre Agassi' 'Andy Roddick' 'Angela Merkel' 'Ann Veneman'
 'Anna Kournikova' 'Ari Fleischer' 'Ariel Sharon' 'Arminio Fraga'
 'Arnoldo Aleman' 'Atal Bihari Vajpayee' 'Benjamin Netanyahu'
 'Bertie Ahern' 'Bill Clinton' 'Bill McBride' 'Bill Simon' 'Bob Graham'
 'Bob Stoops' 'Boris Becker' 'Britney Spears' 'Bulent Ecevit'
 'Calista Flockhart' 'Carlos Menem' 'Celine Dion' 'Cesar Gaviria'
 'Charles Moose' 'Charlton Heston' 'Chen Shui-bian' 'Choi Sung-hong'
 'Christine Todd Whitman' 'Ciro Gomes' 'Colin Farrell' 'Colin Montgomerie'
 'Colin Powell' 'Condoleezza Rice' 'Costas Simitis' 'David Beckham'
 'David Nalbandian' 'David Trimble' 'David Wells' 'Dennis Hastert'
 'Denzel Washington' 'Dick Cheney' 'Donald Rumsfeld' 'Edmund Stoiber'
 'Eduard 

In [8]:
import numpy as np

np.random.seed(42) 
selected_indices = np.random.choice(total_images, size=200, replace=False)
print("kesz")

kesz


In [9]:
selected_images = lfw_dataset.images[selected_indices]
selected_targets = lfw_dataset.target[selected_indices]

In [10]:
selected_names = [lfw_dataset.target_names[t] for t in selected_targets]

In [11]:
import pandas as pd

df = pd.DataFrame({
    'image_index': range(200),
    'person_name': selected_names
})

df.to_csv('selected_images.csv', index=False)
print("A képek hozzárendelése mentve a selected_images.csv fájlba.")

A képek hozzárendelése mentve a selected_images.csv fájlba.


## 2. Feladat

In [13]:
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import matplotlib.pyplot as plt

# Megvizsgáljuk hogy tudunk-e GPU-t használni
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Használt eszköz:", device)

Használt eszköz: cpu


In [14]:
mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20, device=device, post_process=True)

In [15]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [28]:
embeddings = []
names_list = []

for i, img_gray in enumerate(selected_images):
    img_pil = Image.fromarray((img_gray * 255).astype('uint8'), mode='L')
    img_pil = img_pil.convert('RGB')
    
    face = mtcnn(img_pil)
    if face is None:
        print(f"Nincs arc a(z) {i}. képen")
        continue

    with torch.no_grad():
        embedding = resnet(face.unsqueeze(0).to(device))
    embedding = embedding.squeeze().cpu().numpy()

    embeddings.append(embedding)
    names_list.append(selected_names[i])


emb_arr = np.array(embeddings)
name_arr = np.array(names_list)

df_embed = pd.DataFrame(emb_arr)
df_embed['name'] = name_arr
df_embed.to_csv('face_embeddings.csv', index=False)
print("Minden kép feldolgozva, face_embeddings.csv létrehozva.")

Nincs arc a(z) 37. képen
Nincs arc a(z) 52. képen
Nincs arc a(z) 55. képen
Nincs arc a(z) 68. képen
Nincs arc a(z) 71. képen
Nincs arc a(z) 72. képen
Nincs arc a(z) 82. képen
Nincs arc a(z) 84. képen
Nincs arc a(z) 90. képen
Nincs arc a(z) 107. képen
Nincs arc a(z) 111. képen
Nincs arc a(z) 119. képen
Nincs arc a(z) 137. képen
Nincs arc a(z) 153. képen
Nincs arc a(z) 154. képen
Nincs arc a(z) 168. képen
Nincs arc a(z) 174. képen
Nincs arc a(z) 178. képen
Nincs arc a(z) 181. képen
Nincs arc a(z) 183. képen
Nincs arc a(z) 188. képen
Nincs arc a(z) 194. képen
Nincs arc a(z) 197. képen
Minden kép feldolgozva, face_embeddings.csv létrehozva.


In [29]:
emb_arr = np.array(embeddings)
name_arr = np.array(names_list)

df_embed = pd.DataFrame(emb_arr)
df_embed['name'] = name_arr

# A DataFrame utolsó oszlopa lesz a 'name', az első 512 az embedding komponensei.
# Mentés CSV-be:
df_embed.to_csv('face_embeddings.csv', index=False)

print("Face embeddingek és nevek elmentve a face_embeddings.csv fájlba.")

Face embeddingek és nevek elmentve a face_embeddings.csv fájlba.


In [30]:
df_check = pd.read_csv('face_embeddings.csv')
print(df_check.head())

          0         1         2         3         4         5         6  \
0  0.044973  0.004615 -0.002142 -0.025342 -0.069977 -0.004855  0.009609   
1  0.048848 -0.018901 -0.020816 -0.050563 -0.052840 -0.108548 -0.000617   
2  0.035714  0.068572 -0.095392 -0.067740  0.029122 -0.031794 -0.034989   
3  0.038403 -0.021844  0.008826 -0.038153 -0.042987  0.005250  0.009682   
4  0.041577 -0.042635 -0.013814 -0.008284 -0.069091 -0.102482 -0.038224   

          7         8         9  ...       503       504       505       506  \
0  0.033551 -0.030821  0.033543  ... -0.033439  0.018423 -0.008262 -0.036081   
1  0.040050 -0.041165  0.032763  ... -0.007247  0.019450  0.078061 -0.060029   
2  0.115183 -0.029022  0.046971  ... -0.076476  0.044365  0.060939 -0.047252   
3 -0.019798  0.037720 -0.052682  ...  0.025103  0.057463  0.029047  0.051268   
4  0.004350 -0.033183  0.028662  ...  0.014042  0.037236  0.060095 -0.087396   

        507       508       509       510       511             name

In [31]:
print("Beágyazások száma:", len(embeddings))

Beágyazások száma: 177


# 3. Feladat

In [36]:
groups = df_embed.groupby('name')

train_rows = []
test_rows = []

np.random.seed(42)

#Nem tudtam elérni a pontos 150-50 szétválogatást, úgyhogy megközelítőlegesen próbáltam összehozni az arányt.
for name, group in groups:
    group = group.sample(frac=1, random_state=42).reset_index(drop=True)
    
    count = len(group)
    test_count = max(1, round(count * 0.25))
    
    if test_count >= count:
        test_count = count - 1
    
    test_group = group.iloc[:test_count]
    train_group = group.iloc[test_count:]
    
    test_rows.append(test_group)
    train_rows.append(train_group)

# Összefésüljük a train és test halmazokat
train_df = pd.concat(train_rows, ignore_index=True)
test_df = pd.concat(test_rows, ignore_index=True)

print("Train képek száma:", len(train_df))
print("Test képek száma:", len(test_df))

Train képek száma: 141
Test képek száma: 36
Train és Test halmazok létrehozva és elmentve.


# 4-5. Feladat

## Kísérletezés több modellel hogy megnézzük melyik adja a legpontosabb eredményt

### Megoldás döntési fával:

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train = train_df.iloc[:, :-1].values  
y_train = train_df.iloc[:, -1].values   # Az utolsó oszlop a 'name'

X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Pontosság:
acc = accuracy_score(y_test, y_pred)
print("Pontosság a teszt halmazon:", acc)

Pontosság a teszt halmazon: 0.5833333333333334


### KNN modell

In [44]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Pontosság:
acc = accuracy_score(y_test, y_pred)
print("Pontosság a teszt halmazon (KNN):", acc)

Pontosság a teszt halmazon (KNN): 0.7777777777777778


### Random Forest:

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Pontosság:
acc = accuracy_score(y_test, y_pred)
print("Pontosság a teszt halmazon (Random Forest):", acc)

Pontosság a teszt halmazon (Random Forest): 0.9166666666666666


### A három model közül kijelenthetjük hogy a Random Forest model adja a legpontosabb eredményt, de a KNN modell is jól teljesít.